In [0]:
# mount drive, to use the uploaded dataset

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# install dependencies

# from scipy.misc import imread, imsave
# ImportError: cannot import name 'imread'
# => downgrade scipy
!pip3 install scipy==1.1.0 

#!pip3 install torch==1.0.0
#!pip3 install torchvision==0.2.1

     |████████████████████████████████| 31.2MB 98kB/s 
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [0]:
# clone code

!git clone https://github.com/simonedeldeo/DAIN.git
%cd DAIN

Cloning into 'DAIN'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 613 (delta 8), reused 6 (delta 3), pack-reused 587
Receiving objects: 100% (613/613), 188.56 KiB | 754.00 KiB/s, done.
Resolving deltas: 100% (336/336), done.
/content/DAIN


In [0]:
# build code

%%capture
%cd my_package 
!./build.sh
%cd ..
%cd PWCNet/correlation_package_pytorch1_0
!./build.sh
%cd ..
%cd ..

In [0]:
# get dataset1

%%capture
!rm -rf val
!cp '/content/drive/My Drive/__progetto int sys/data/val - Star360.zip' './'
!mkdir val
!unzip 'val - Star360.zip' -d val

In [0]:
# get dataset2

%%capture
!rm -rf AniDSval_360
!cp '/content/drive/My Drive/__progetto int sys/data/AniDSval_360.zip' './'
!unzip 'AniDSval_360.zip'

In [0]:
# preprocess dataset

!rm -rf MiddleBurySet
!mkdir MiddleBurySet
!mkdir MiddleBurySet/other-data
!mkdir MiddleBurySet/other-gt-interp
!mkdir MiddleBurySet/other-result-author

import os

# AniDSval_360
basepath = '/content/DAIN/val'
data_path = 'MiddleBurySet/other-data/'
gt_path = 'MiddleBurySet/other-gt-interp/'
sub_f = os.listdir(basepath)
sub_f.sort()
for s in sub_f:
  path = os.path.join(basepath, s)
  os.makedirs(os.path.join(data_path, s))
  os.makedirs(os.path.join(gt_path, s))
  os.rename(os.path.join(path, '0.jpg'), os.path.join(os.path.join(data_path, s), 'frame0.jpg'))
  os.rename(os.path.join(path, '1.jpg'), os.path.join(os.path.join(gt_path, s), 'frame1.jpg'))
  os.rename(os.path.join(path, '2.jpg'), os.path.join(os.path.join(data_path, s), 'frame2.jpg'))

In [0]:
# get pretrained model

!rm -rf model_weights
!mkdir model_weights
%cd model_weights
#!wget http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth
!cp '/content/drive/My Drive/__progetto int sys/ckps/DAIN/4k_720/64307-Sun-Feb-23-01:35/epoch1.pth' 'best.pth'
%cd ..

/content/DAIN/model_weights
/content/DAIN


In [0]:
# generate interpolated images

!CUDA_VISIBLE_DEVICES=0 python -W ignore demo_MiddleBury.py

revise the unique id to a random numer 23979
Namespace(SAVED_MODEL=None, alpha=[0.0, 1.0], arg='./model_weights/23979-Sun-Feb-23-11:16/args.txt', batch_size=1, channels=3, ctx_lr_coe=1.0, datasetName='Vimeo_90K_interp', datasetPath='', dataset_split=97, debug=False, depth_lr_coe=0.001, dtype=<class 'torch.cuda.FloatTensor'>, epsilon=1e-06, factor=0.2, filter_lr_coe=1.0, filter_size=4, flow_lr_coe=0.01, force=False, log='./model_weights/23979-Sun-Feb-23-11:16/log.txt', lr=0.002, netName='DAIN', no_date=False, numEpoch=100, occ_lr_coe=1.0, patience=5, rectify_lr=0.001, save_path='./model_weights/23979-Sun-Feb-23-11:16', save_which=1, seed=1, time_step=0.5, uid=None, use_cuda=True, use_cudnn=1, weight_decay=0, workers=8)
cudnn is used
The testing model weight is: ./model_weights/best.pth
The unique id for current testing is: 96008
38
*****************current image process time 	 12.411336421966553s ******************
interpolation error / PSNR : 2.723 / 28.8614
The average interpolation e

In [0]:
# get stats

import PIL
import PIL.Image
import skimage
import skimage.measure
import numpy
import math

gt = '/frame1.jpg'
interp = '/frame1.png'

basepath1 = './MiddleBurySet/other-gt-interp/'
id = '96008'
basepath2 = './MiddleBurySet/other-result-author/' + id + '/'
sub_f = os.listdir(basepath1)
sub_f.sort()
dblPsnr = []
dblSsim = []
dblIE = []
for s in sub_f:
  path1 = os.path.join(basepath1, s)
  path2 = os.path.join(basepath2, s)
  dblPsnr.append(skimage.measure.compare_psnr(im_true=numpy.array(PIL.Image.open(path1 + gt))[:, :, ::-1], im_test=numpy.array(PIL.Image.open(path2 + interp))[:, :, ::-1], data_range=255))
  dblSsim.append(skimage.measure.compare_ssim(X=numpy.array(PIL.Image.open(path1 + gt))[:, :, ::-1], Y=numpy.array(PIL.Image.open(path2 + interp))[:, :, ::-1], data_range=255, multichannel=True))
  dblIE.append(math.sqrt(skimage.measure.compare_mse(numpy.array(PIL.Image.open(path1 + gt))[:, :, ::-1], numpy.array(PIL.Image.open(path2 + interp))[:, :, ::-1])))

print('computed average psnr', numpy.mean(dblPsnr))
print('computed average ssim', numpy.mean(dblSsim))
print('computed average ie', numpy.mean(dblIE))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: DEPRECATED: skimage.measure.compare_psnr has been moved to skimage.metrics.peak_signal_noise_ratio. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: DEPRECATED: skimage.measure.compare_mse has been moved to skimage.metrics.mean_squared_error. It will be removed from skimage.measure in version 0.18.


computed average psnr 24.006718201485622
computed average ssim 0.8767172284745116
computed average ie 17.375263539795252
